In [2]:
import win32com.client  # Windows COM 인터페이스 사용
import xml.etree.ElementTree as ET


# VISSIM 실행 및 연결
Vissim = win32com.client.Dispatch("Vissim.Vissim")  # Vissim 인스턴스 생성

# VISSIM 네트워크 파일 (.inpx) 로드
network_path = r"C:\VISSIM_Workspace\test\test.inpx"  # 실제 파일 경로 입력

Vissim.LoadNet(network_path)


# Vehicle Input 리스트 가져오기
vehicle_inputs = Vissim.Net.VehicleInputs.GetAll()


count = len(vehicle_inputs)

# 값 가져오기
for i in range(count):
    link_number = i+1
    link_name = Vissim.Net.VehicleInputs.ItemByKey(link_number).AttValue('Name')
    volume = Vissim.Net.VehicleInputs.ItemByKey(link_number).AttValue('Volume(1)') # Volumn은 1 고정
    print(f"Link {link_number} - Link_name {link_name} - Volume: {volume}")

# 변수 리스트 설정
new_names = ["a", "b"]
new_volume = ["100", "200"]

# 값 변경
for i in range(count):
  Vissim.Net.VehicleInputs.ItemByKey(i+1).SetAttValue('Name', new_names[i])
  Vissim.Net.VehicleInputs.ItemByKey(i+1).SetAttValue('Volume(1)', new_volume[i])

for i in range(count):
    link_number = i+1
    link_name = Vissim.Net.VehicleInputs.ItemByKey(link_number).AttValue('Name')
    volume = Vissim.Net.VehicleInputs.ItemByKey(link_number).AttValue('Volume(1)') # Volumn은 1 고정
    print(f"Link {link_number} - Link_name {link_name} - Volume: {volume}")

print("VISSIM 시뮬레이션 실행 중...")
Vissim.Simulation.RunContinuous()
#
Vissim.Simulation.Stop()
print("VISSIM 시뮬레이션 종료")


# XML 파일 파싱
tree = ET.parse(network_path)
root = tree.getroot()

interval_value = 0
simPeriod_value = int(Vissim.Simulation.AttValue("SimPeriod"))  # 전체 시뮬레이션 실행 시간
# dataColl 태그에서 interval 속성 값 찾기
for elem in root.iter("dataColl"):
    interval_value = int(elem.get("interval"))  # "interval" 속성 값 가져오기
    if interval_value:
        print(f"추출된 interval 값: {interval_value}") # 60
    else:
        print("추출된 interval 값이 없습니다.")


data_collection = Vissim.Net.DataCollectionMeasurements.GetAll()
data_length = int(len(data_collection))

num_time_intervals = int(simPeriod_value / interval_value)
print("num_time_intervals : ", num_time_intervals)

Sim_run = Vissim.Simulation.AttValue("SimRes")
print("Sim_run : ", Sim_run)

for i in range(1,num_time_intervals+1):
    for j in range(1, data_length+1):

        No_Veh = Vissim.Net.DataCollectionMeasurements.ItemByKey(j).AttValue(f"Vehs        (Avg,{i},All)") # number of vehicles(차량 대수)
        if No_Veh:
            No_Veh = round(No_Veh,2)
        Speed = Vissim.Net.DataCollectionMeasurements.ItemByKey(j).AttValue(f"Speed        (Avg,{i},All)") # number of vehicles(차량 대수)
        if Speed:
            Speed = round(Speed,2)
        Length = Vissim.Net.DataCollectionMeasurements.ItemByKey(j).AttValue(f"Length        (Avg,{i},All)") # number of vehicles(차량 대수)
        if Length:
            Length = round(Length,2)
        print("No_Veh : ", No_Veh)
        print("Speed : ", Speed)
        print("Length : ", Length)


# Vissim.Exit()  # VISSIM 종료



KeyboardInterrupt: 